In [1]:
!pip uninstall sktime -y

Found existing installation: sktime 0.32.1
Uninstalling sktime-0.32.1:
  Successfully uninstalled sktime-0.32.1


In [2]:
# !pip install ../.. --upgrade --no-cache-dir
!pip install git+https://github.com/thierrymoudiki/sktime.git@nnetsauce-mts --upgrade --no-cache-dir

  Cloning https://github.com/thierrymoudiki/sktime.git (to revision nnetsauce-mts) to /tmp/pip-req-build-qz9j3m9n
  Running command git clone --filter=blob:none --quiet https://github.com/thierrymoudiki/sktime.git /tmp/pip-req-build-qz9j3m9n
  Running command git checkout -b nnetsauce-mts --track origin/nnetsauce-mts
  Switched to a new branch 'nnetsauce-mts'
  Branch 'nnetsauce-mts' set up to track remote branch 'nnetsauce-mts' from 'origin'.
  Resolved https://github.com/thierrymoudiki/sktime.git to commit f98cff0253117dfc09eeebfdaeaebf02bbb35e4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sktime: filename=sktime-0.32.1-py3-none-any.whl size=29312168 sha256=2e69d637fef653cb3ce15c64f3ff7fffcdc78e713811aa5ff3db5cd455d9cc5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-toc5p3xc/wheels/8f/6f/81/505b6cf562ee44b6a30237eba488bd229778175235b27d5e95
Successfully built sktime


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
from statsmodels.tsa.base.datetools import dates_from_str

from sktime.forecasting.nnetsaucemts import MTS

In [4]:
# some example data
mdata = sm.datasets.macrodata.load_pandas().data
# prepare the dates index
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
print(mdata.head())
mdata = mdata[['realgovt', 'tbilrate', 'cpi']]
mdata.index = pd.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()

n = data.shape[0]
max_idx_train = np.floor(n*0.9)
training_index = np.arange(0, max_idx_train)
testing_index = np.arange(max_idx_train, n)
df_train = data.iloc[training_index,:]
df_test = data.iloc[testing_index,:]

     year  quarter  realgdp  realcons  realinv  realgovt  realdpi   cpi  \
0 1959.00     1.00  2710.35   1707.40   286.90    470.05  1886.90 28.98   
1 1959.00     2.00  2778.80   1733.70   310.86    481.30  1919.70 29.15   
2 1959.00     3.00  2775.49   1751.80   289.23    491.26  1916.40 29.35   
3 1959.00     4.00  2785.20   1753.70   299.36    484.05  1931.30 29.37   
4 1960.00     1.00  2847.70   1770.50   331.72    462.20  1955.50 29.54   

      m1  tbilrate  unemp    pop  infl  realint  
0 139.70      2.82   5.80 177.15  0.00     0.00  
1 141.70      3.08   5.10 177.83  2.34     0.74  
2 140.50      3.82   5.30 178.66  2.74     1.09  
3 140.00      4.33   5.60 179.39  0.27     4.06  
4 139.60      3.50   5.20 180.01  2.31     1.19  


In [5]:
# Adjust Ridge
regr = linear_model.Ridge()
obj_MTS = MTS(regr,
              lags = 25,
              a = 0.1,
              n_hidden_features=5,
              verbose = 1,
              type_pi="gaussian")
display(obj_MTS.get_params())

{'a': 0.1,
 'activation_name': 'relu',
 'agg': 'mean',
 'backend': 'cpu',
 'bias': True,
 'block_size': None,
 'cluster_encode': True,
 'direct_link': True,
 'dropout': 0,
 'kernel': 'gaussian',
 'lags': 25,
 'n_clusters': 2,
 'n_hidden_features': 5,
 'nodes_sim': 'sobol',
 'obj': Ridge(),
 'replications': None,
 'seed': 123,
 'show_progress': True,
 'type_clust': 'kmeans',
 'type_pi': 'gaussian',
 'type_scaling': ('std', 'std', 'std'),
 'verbose': 1,
 'obj__alpha': 1.0,
 'obj__copy_X': True,
 'obj__fit_intercept': True,
 'obj__max_iter': None,
 'obj__positive': False,
 'obj__random_state': None,
 'obj__solver': 'auto',
 'obj__tol': 0.0001}

## 1 - Almost works

Why specifying `fh` here, while declared as `None` in `_fit`?

In [6]:
obj_MTS.fit(df_train, fh=[1, 2, 3, 4, 5])


 Adjusting Ridge to multivariate time series... 
 


100%|██████████| 3/3 [00:00<00:00, 196.01it/s]


MTS(a=0.1, lags=25, obj=Ridge(), type_pi='gaussian', verbose=1)

In [7]:
# with Gaussian prediction intervals
res = obj_MTS.predict(fh=[1, 2, 3, 4, 5])
print(res)

TypeError: No valid mtype could be identified for object of type <class 'pandas.core.frame.DataFrame'>. Errors returned are as follows, in format [mtype]: [error message] 
pd.DataFrame: <class 'pandas.core.indexes.base.Index'> is not supported for obj, use one of (<class 'pandas.core.indexes.range.RangeIndex'>, <class 'pandas.core.indexes.period.PeriodIndex'>, <class 'pandas.core.indexes.datetimes.DatetimeIndex'>) or integer index instead.
pd.Series: obj must be a pandas.Series, found <class 'pandas.core.frame.DataFrame'>
np.ndarray: obj must be a numpy.ndarray, found <class 'pandas.core.frame.DataFrame'>
xr.DataArray: obj must be a xarray.DataArray, found <class 'pandas.core.frame.DataFrame'>
dask_series: obj must be a dask DataFrame, found <class 'pandas.core.frame.DataFrame'>
pl.DataFrame: obj must be a polars DataFrame, found <class 'pandas.core.frame.DataFrame'>


# 2 - Doesn't work

Why specifying `fh` here, while declared as `None`?

In [8]:
obj_MTS.fit(df_train)

ValueError: The forecasting horizon `fh` must be passed to `fit` of MTS, but none was found. This is because fitting of the forecaster MTS depends on `fh`. 